In [39]:
from pathlib import Path
from typing import List, NamedTuple
from pydantic import BaseModel


class ShowMetadata(NamedTuple):
    showname: str
    dates_filepath: Path
    titles_filepath: Path


class ShowMetadataList(BaseModel):
    shows: List[ShowMetadata]


def meta_dict(file_path):
    file = open(file_path, "r").read().splitlines()
    return {
        key: value
        for key, value in [
            (line.split(" - ")[0], line.split(" - ")[1]) for line in file
        ]
    }


def metadata_to_dict(metadata: ShowMetadataList):
    metadata_dict = {}
    for show in metadata.shows:
        showname, dates_filepath, titles_filepath = show
        metadata_dict[showname] = {
            "dates": meta_dict(dates_filepath),
            "titles": meta_dict(titles_filepath),
        }
    return metadata_dict

In [ ]:
data = ShowMetadataList(
    shows=[
        ShowMetadata(
            "rotl",
            Path("../files/meta/rotl_dates.txt"),
            Path("../files/meta/rotl_titles.txt"),
        ),
        ShowMetadata(
            "roadwork",
            Path("../files/meta/roadwork_dates.txt"),
            Path("../files/meta/roadwork_titles.txt"),
        ),
    ]
)

metadata = metadata_to_dict(data)

In [ ]:
from pathlib import Path

root_dir = Path("../files/summaries")

files = [file for file in root_dir.rglob("*") if file.is_file()]
for file in files:
    out_dir = Path("../files/indexes")
    showname = str(Path(file.relative_to(root_dir)).parent)
    episode = file.stem.split("_-_")[0]
    title = metadata[showname]["titles"][episode]
    date = metadata[showname]["dates"][episode]